In [1]:
import pandas as pd
import muon as mu
import scanpy as sc
from muon import atac as ac
import numpy as np
import warnings
import pychromvar as pc
import sys
import os
import pickle


/root/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_dir = os.path.abspath('')
data_path = os.path.join(file_dir, '..', '..', 're_design', '10x_data')

h5_file_path = os.path.join(data_path, 'pbmc3k_multi.h5mu')
save_path = os.path.join(file_dir, 'generated_data')

In [3]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    mdata = mu.read_h5mu(h5_file_path)
mdata

MuData object with n_obs × n_vars = 3217 × 32907
  obs:	'leiden_wnn'
  uns:	'leiden', 'leiden_wnn_colors', 'umap', 'wnn'
  obsm:	'X_umap', 'X_wnn_umap'
  obsp:	'wnn_connectivities', 'wnn_distances'
  3 modalities
    gene:	3217 x 15863
      obs:	'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden_umap', 'leiden_euclidean', 'leiden_wnn', 'tf_leiden_umap', 'tf_leiden_euclidean'
      var:	'gene_ids', 'feature_types', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std', 'gene_ids_simple', 'tf'
      uns:	'hvg', 'leiden', 'leiden_umap_colors', 'leiden_umap_sizes', 'log1p', 'neighbors_euclidean', 'neighbors_umap', 'paga', 'pca', 'rank_loci_groups', 'tf_neighbors_euclidean', 'tf_neighbors_umap', 'umap'
      obsm:	'X_pca', 'X_umap', 'tf_pca'
      varm:	'PCs'
      layers:	'counts'
      obsp:	'neighbors_euclidean_connectivities', 'neighbors_euclidean_distances', 'neighbors_umap_connectivities', 'neighbors_umap_distances', 'tf_neighbors_umap_connectivities', 'tf_neighbors_umap_distances'
    cre:	3217 x 16298
      obs:	'n_genes_by_counts', 'total_counts', 'avg_frag_length', 'leiden_umap', 'leiden_umap20', 'leiden_umap50', 'leiden_euclidean', 'leiden_wnn'
      var:	'directionality', 'orientation', 'bias_strand', 'fwd_count', 'rev_count', 'chrom', 'summit_center', 'abs_summit_diff', 'oritn_summit_diff', 'start', 'end', 'strand', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'frag_length', 'gc_bias'
      uns:	'hvg', 'leiden', 'leiden_euclidean_colors', 'leiden_umap_colors', 'leiden_umap_sizes', 'log1p', 'motif_name', 'neighbors_euclidean', 'neighbors_umap', 'paga', 'pca', 'peak_seq', 'rank_loci_groups', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs', 'bg_peaks', 'motif_match'
      layers:	'counts'
      obsp:	'neighbors_euclidean_connectivities', 'neighbors_euclidean_distances', 'neighbors_umap_connectivities', 'neighbors_umap_distances'
    chromvar:	3217 x 746
      obs:	'leiden_wnn'
      uns:	'rank_genes_groups'

# Differentially accessible peaks based on Leiden clustering on tCRE

In [5]:
# rank the differentially expressed genes  
sc.tl.rank_genes_groups(mdata.mod['cre'], 'leiden_umap', method='t-test')

In [31]:
# transform into format peak,cluster and filter by p-value and take only first n
from collections import defaultdict

rank_peak_groups = mdata.mod['cre'].uns['rank_genes_groups']
filter_by_pvalue = pd.DataFrame(rank_peak_groups['pvals_adj'])<0.05
de_cre = pd.DataFrame(rank_peak_groups['names'])[filter_by_pvalue].iloc[:10000,:]
pvalues = pd.DataFrame(rank_peak_groups['pvals_adj'])[filter_by_pvalue].iloc[:10000,:]
cre_cluster_pvalues  = defaultdict(dict)
# flatten into list of unique genes in format gene:cluster 
for idx, row in de_cre.iterrows():
    for cluster, peak in enumerate(row):
        if not pd.isna(peak) and cluster not in cre_cluster_pvalues[peak]:
            cre_cluster_pvalues[peak][cluster] = pvalues.iloc[idx,cluster]

cre_cluster = {}

for peak, cluter_dict in cre_cluster_pvalues.items():
    # choose cluster with minimum p-value
    cre_cluster[peak] = min(cluter_dict, key=cluter_dict.get)

# Final dataset

In [32]:
mdata['cre'].var['peak_seq'] = mdata.mod['cre'].uns['peak_seq']
full_data = mdata['cre'].var.reset_index(names='interval')
# subset only required columns
full_data = full_data[['interval','chrom','start','end','summit_center','peak_seq']]

# new column cluster by mapping genes to cluster with help of gene_cluster dict
full_data['cell_type'] = full_data['interval'].map(cre_cluster)

# full_data = full_data.explode('cell_type')
full_data = full_data[full_data.cell_type.notna()]

full_data['cell_type'] = "ct"+(full_data.cell_type.astype(int)).astype(str)
full_data.reset_index(drop=True,inplace=True)
# for data constitencty with legacy code
full_data.rename(columns={'peak_seq':'sequence', 'interval':'peak'}, inplace=True)

full_data = full_data.drop(full_data[full_data.sequence.str.contains("N")].index).reset_index(drop=True)
full_data

,peak,chrom,start,end,summit_center,sequence,cell_type
0,chr1_29236_29737_-,chr1,29236,29737,29337,CTCCCTCCAGCCCCTCCGGGTCCCCTACTTCGCCCCGCCAGGCCCC...,ct11
1,chr1_629102_629950_+,chr1,629102,629950,629793,AGGCTTCAACATCGAATACGCCGCAGGCCCCTTCGCCCTATTCTTC...,ct0
2,chr1_630597_631171_+,chr1,630597,631171,630948,CAAAACCCACCCCATTCCTCCCCACACTCATCGCCCTTACCACACT...,ct1
3,chr1_632356_633841_+,chr1,632356,633841,633051,AACACTTTCTCGGCCTATCCGGAATGCCCCGACGTTACTCGGACTA...,ct0
4,chr1_634002_634689_+,chr1,634002,634689,634385,CCATGGCCATCCCCTTATGAGCGGGCGCAGTGATTATAGGCTTTCG...,ct0
...,...,...,...,...,...,...,...
14625,chrY_21028687_21029188_-,chrY,21028687,21029188,21028788,TGTGGTGAGTGTTATAGCTCATAATGTTGGCACGGACCCAAACAGT...,ct11
14626,chrY_21138232_21138733_+,chrY,21138232,21138733,21138531,ACATGACTTGCATATTTAGCATGTTAACTGCTTCATTTGGGGAGCT...,ct11
14627,chrY_21254335_21254836_+,chrY,21254335,21254836,21254594,AAATAATAAAGTGTATTATTTATCTGTTTTACATACTGTTGGTTTT...,ct11
14628,chrY_57067464_57067965_+,chrY,57067464,57067965,57067865,CGAAAGTAGAGGCAGTTCCTGTCAGATGAATTCTATTTTGTCTGTG...,ct1


In [38]:
# check for nan values
full_data.isna().any()

peak             False
chrom            False
start            False
end              False
summit_center    False
sequence         False
cell_type        False
dtype: bool

In [39]:
print("Total peaks:", full_data.shape[0],"; more than 1 cluster:", full_data[full_data['peak'].duplicated()].shape[0])

Total peaks: 14630 ; more than 1 cluster: 0


In [40]:
print("File statistics")
print(full_data['cell_type'].value_counts())
print(full_data['chrom'].value_counts())

File statistics
cell_type
ct11    7403
ct1     2574
ct0     2073
ct3      724
ct2      514
ct8      290
ct6      244
ct7      227
ct4      226
ct10     140
ct5      130
ct9       85
Name: count, dtype: int64
chrom
chr1     1438
chr19    1109
chr2     1005
chr17     946
chr11     811
chr6      789
chr7      771
chr12     761
chr3      760
chr16     658
chr5      653
chr14     564
chr9      542
chr10     531
chr4      504
chr8      473
chr15     459
chrX      415
chr20     386
chr22     359
chr13     252
chr18     202
chr21     189
chrY       36
chrM       17
Name: count, dtype: int64


In [41]:
full_data['sequence'].str.len().describe()

count    14630.000000
mean       559.814491
std        150.576832
min        501.000000
25%        501.000000
50%        501.000000
75%        501.000000
max       3654.000000
Name: sequence, dtype: float64

In [42]:
full_data[full_data['chrom']=='chr1']['cell_type'].value_counts()

cell_type
ct11    688
ct1     269
ct0     220
ct3      84
ct2      51
ct8      26
ct6      25
ct7      23
ct4      22
ct5      13
ct10     13
ct9       4
Name: count, dtype: int64

In [43]:
file_path = os.path.join(save_path, 'tcre_seq_leiden_cluster_12.csv')

full_data.to_csv(file_path, index=False)

# Explore

In [47]:
full_data = pd.read_csv(f'{save_path}/tcre_seq_leiden_cluster.csv')
full_data

,peak,chrom,start,end,summit_center,sequence,cell_type
0,chr1_29236_29737_-,chr1,29236,29737,29337,CTCCCTCCAGCCCCTCCGGGTCCCCTACTTCGCCCCGCCAGGCCCC...,ct2
1,chr1_199774_200275_-,chr1,199774,200275,200006,CCCTACTTCGCCCCGCCAGGCCCCCACGACCCTACTTCCCGCGGCC...,ct1
2,chr1_199774_200275_-,chr1,199774,200275,200006,CCCTACTTCGCCCCGCCAGGCCCCCACGACCCTACTTCCCGCGGCC...,ct2
3,chr1_629102_629950_+,chr1,629102,629950,629793,AGGCTTCAACATCGAATACGCCGCAGGCCCCTTCGCCCTATTCTTC...,ct2
4,chr1_630597_631171_+,chr1,630597,631171,630948,CAAAACCCACCCCATTCCTCCCCACACTCATCGCCCTTACCACACT...,ct1
...,...,...,...,...,...,...,...
14800,chrY_20575222_20575876_+,chrY,20575222,20575876,20575706,GTCAGGGGTTTGAGAGCCTGGCCAACATGGTGAAACCCCATCTCTA...,ct2
14801,chrY_21028687_21029188_-,chrY,21028687,21029188,21028788,TGTGGTGAGTGTTATAGCTCATAATGTTGGCACGGACCCAAACAGT...,ct2
14802,chrY_21138232_21138733_+,chrY,21138232,21138733,21138531,ACATGACTTGCATATTTAGCATGTTAACTGCTTCATTTGGGGAGCT...,ct0
14803,chrY_21254335_21254836_+,chrY,21254335,21254836,21254594,AAATAATAAAGTGTATTATTTATCTGTTTTACATACTGTTGGTTTT...,ct0


In [48]:
data = full_data.drop(full_data[full_data.sequence.str.contains("N")].index).reset_index(drop=True)
data['sequence'] = data['sequence'].str[-200:]
data

,peak,chrom,start,end,summit_center,sequence,cell_type
0,chr1_29236_29737_-,chr1,29236,29737,29337,ACTCCGAGCTCCCGACGTGCACACGGCTCCCATGCGTTGTCTTCCG...,ct2
1,chr1_199774_200275_-,chr1,199774,200275,200006,ACGGCTCCCATGCGTTGTCTTCCGAGCGTCAGGCCGCCCCTACCCG...,ct1
2,chr1_199774_200275_-,chr1,199774,200275,200006,ACGGCTCCCATGCGTTGTCTTCCGAGCGTCAGGCCGCCCCTACCCG...,ct2
3,chr1_629102_629950_+,chr1,629102,629950,629793,CTAGCTTTTATTCCAGTTCTAACCAAAAAAATAAACCCTCGTTCCA...,ct2
4,chr1_630597_631171_+,chr1,630597,631171,630948,TCAATATGAAAATCACCTCAGAGCTGGTAAAAAGAGGCTTAACCCC...,ct1
...,...,...,...,...,...,...,...
14800,chrY_20575222_20575876_+,chrY,20575222,20575876,20575706,AGGCGGGGAAAAGCATCGTAATCAGCTGCGTCGCCTTTTGGTGACG...,ct2
14801,chrY_21028687_21029188_-,chrY,21028687,21029188,21028788,GATAGATAGAAAAGTTATCCCAGTCCCCACCCAAACCAGAAGCCCA...,ct2
14802,chrY_21138232_21138733_+,chrY,21138232,21138733,21138531,GGAGTTGCACACACAGGTTTACTGATAAGAGAAGTTACTCAAACTG...,ct0
14803,chrY_21254335_21254836_+,chrY,21254335,21254836,21254594,TTTTTGTCGGGTGGAAGCATGAATACTTGTTATTCAAGTGTTCAGG...,ct0


In [49]:
train_data = data[(data["chrom"]!= "chr1") & (data["chrom"] != "chr2")].reset_index(drop=True)
train_data['cell_type'].value_counts()

cell_type
ct2    5007
ct1    4494
ct0    2823
Name: count, dtype: int64

In [50]:
test_data = data[data['chrom'] == "chr1"].reset_index(drop=True)
test_data['cell_type'].value_counts()

cell_type
ct2    579
ct1    569
ct0    336
Name: count, dtype: int64